In [17]:
import cobra
from cobra.io import read_sbml_model
from pprint import pprint
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from cobra.medium import minimal_medium

from process_bigraph import Composite
from process_bigraph import ProcessTypes
from process_bigraph.emitter import gather_emitter_results

from cdFBA import register_types
from cdFBA.processes.dfba import dFBA, UpdateEnvironment, StaticConcentration, Injector, WaveFunction

from cdFBA.utils import make_cdfba_composite, get_injector_spec, get_wave_spec, get_static_spec, set_concentration, set_kinetics, get_exchanges

from in_silico_functions.carbon_sources import (load, met_info, find_fluxes, make_fluxes_dict, make_dict,main, classify_met)

In [2]:
def get_mini_medium(model, target_growth=10e-4):
    ## make a copy of the model
    try_model = model.copy()

    ## compute the minimum media neccessary for growth at targeted_growth
    mini_growth = minimal_medium(try_model, target_growth, minimize_components=10, open_exchanges=True)

    ## set minimal medium
    mini_medium = {}
    for i in mini_growth.index:
        mini_medium[i] = try_model.medium[i]

    ## get all other metabolites that can be added to the medium
    additional_medium = try_model.medium
    for i in mini_medium.keys():
        del additional_medium[i]
    return mini_medium, additional_medium

In [3]:
# Load models

gut_models = {
        "B_thetaiotaomicron": "/Users/rebekahsheih/PycharmProjects/Rebekahs_Rotation_Project/Vivarium/cdFBA-main/Notebooks/sbml/Bacteroides_thetaiotaomicron_VPI_5482.xml",
        "E_rectale": "/Users/rebekahsheih/PycharmProjects/Rebekahs_Rotation_Project/Vivarium/cdFBA-main/Notebooks/sbml/Eubacterium_rectale_ATCC_33656.xml"
    }

models = {k: read_sbml_model(v) for k, v in gut_models.items()}

BT = models["B_thetaiotaomicron"]
ER = models["E_rectale"]

In [4]:
# Create minimal media
medium_bt = get_mini_medium(BT)[0]
medium_bt["EX_MGlcn175_rl(e)"] = 0.00
medium_bt["EX_cholate(e)"] = 1000.00 # want bt to consume cholate and produce acetate for er to eat

BT.medium = medium_bt

medium_er = get_mini_medium(ER)[0]
medium_er["EX_pullulan1200(e)"] = 10e-6 # if this number is small enough, er produces more butyrate
medium_er["EX_ac(e)"] = 0 # want ER to consume acetate from bt, so don't give it acetate.
# I am seeing that organism consumes acetate and pullulan together, but does not consume acetate unless there is enough pullulan in media.
# w/o acetate, uses more pullulan1200, w/acetate, consumes both

ER.medium = medium_er

In [18]:
# write minimal media files

cobra.io.write_sbml_model(BT, "sbml/bt.xml")
bt_try = read_sbml_model("sbml/bt.xml")
bt_try.optimize()
print(bt_try.summary())

cobra.io.write_sbml_model(ER, "sbml/er.xml")
er_try = read_sbml_model("sbml/er.xml")
er_try.optimize()
print(er_try.summary())

gut_models_mini = {
        "B_thetaiotaomicron": "/Users/rebekahsheih/PycharmProjects/Rebekahs_Rotation_Project/in_silico_scripts/sbml/bt.xml",
        "E_rectale": "/Users/rebekahsheih/PycharmProjects/Rebekahs_Rotation_Project/in_silico_scripts/sbml/er.xml"
    }

models = {k: read_sbml_model(v) for k, v in gut_models_mini.items()}

OSError: The file with 'sbml/bt.xml' does not exist, or is not an SBML string. Provide the path to an existing SBML file or a valid SBML string representation:


In [ ]:
exchanges = ['EX_ac(e)', 'EX_but(e)', "'EX_ch4(e)','EX_cholate(e)'] # acetate, butyrate, H2, methane, cholate

volume = 2

# dFBA model
spec = make_cdfba_composite(gut_models_mini, medium_type=None, exchanges=exchanges, volume=volume, interval=0.1)

pprint(spec)

In [ ]:
#Set reaction bounds (constrain bounds here to make it so that the exchange reactions can't take place)
spec['Species']['B_thetaiotaomicron']['config']['bounds'] = {
            "EX_o2(e)": {"lower": -2, "upper": None},
            "DM_atp_c_": {"lower": 1, "upper": 1},
        }
spec['Species']['E_rectale']['config']['bounds'] = {
            "EX_o2(e)": {"lower": -2, "upper": None},
            "DM_atp_c_": {"lower": 1, "upper": 1}
        }

In [ ]:
#set external substrate concentrations
concentrations = {      # zero polysaccharides (pectin and cholate) to stop b theta from growing, limit acetate so that it has to come from keystone species
    'acetate': 0,
    'butyrate': 0,
    'Methane': 0,
    'Cholate' : 10 # give cholate so that BT can grow and produce acetate for ER

}
set_concentration(spec, concentrations)

In [ ]:
#set kinetics
kinetics = {
    'acetate': (0.5, 5), # reduce km, increase Vmax
    'butyrate': (0.5, 5),
    'Methane': (0.5, 5),
    'Cholate': (0.5, 5)
}
for species in gut_models.keys():
    set_kinetics(species, spec, kinetics)
pprint(spec)

In [ ]:
#set emitter specs
spec['emitter'] = {
        "_type": "step",
        "address": "local:ram-emitter",
        "config": {
            "emit": {
                "shared_environment": "any",
                "global_time": "any",
            }
        },
        "inputs": {
            "shared_environment": ["Shared Environment"],
            "global_time": ["global_time"]
        }
    }

In [ ]:
#create the core object
core = ProcessTypes()
#register data types
core = register_types(core)
#register all processes and steps
core.register_process('dFBA', dFBA)
core.register_process('UpdateEnvironment', UpdateEnvironment)
core.register_process('StaticConcentration', StaticConcentration)
core.register_process('WaveFunction', WaveFunction)
core.register_process('Injector', Injector)

In [ ]:
# KO all reactions for one species

In [ ]:
sim = Composite({
        "state": spec,
        },
        core=core
    )

In [ ]:
#run simulation
sim.run(20)

In [ ]:
#gather results
results = gather_emitter_results(sim)[('emitter',)]

In [ ]:
#extract time-series data
timepoints = []
for timepoint in results:
    time = timepoint.pop('global_time')
    timepoints.append(time)
env = [timepoint['shared_environment']['concentrations'] for timepoint in results]
env_combined = {}
for d in env:
    for key, value in d.items():
        if key not in env_combined:
            env_combined[key] = []
        env_combined[key].append(value)

In [ ]:
results

In [ ]:
#plot results for biomass
fig, ax = plt.subplots(dpi=300)
for key, value in env_combined.items():
    if key not in ['acetate', 'pectins', 'butyrate', 'Methane', 'Cholate']:
        ax.plot(timepoints, env_combined[key], label=key)
        ax.set_yscale('log')
plt.xlabel('Time (h)')
plt.ylabel('Biomass (gDW)')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
#plot substrates
fig, ax = plt.subplots(dpi=300)
for key, value in env_combined.items():
    if key in ['acetate', 'Cholate']:
        ax.plot(timepoints, env_combined[key], label=key)
        ax.set_yscale('log')
plt.xlabel('Time (h)')
plt.ylabel('Substrate Concentration (mM)')
plt.legend()
plt.tight_layout()
plt.show()